In [1]:
!pip install outlines pydantic transformers -q && pip freeze | grep -w "outlines\|pydantic\|transformers"

outlines==0.0.8
pydantic==2.2.0
transformers==4.32.0


In [2]:
import json
from enum import Enum
from functools import partial
from typing import Optional

from pydantic import BaseModel

from outlines import models
from outlines import text
from outlines.text import generate

In [3]:
# load a small language model

model = models.transformers("gpt2")

In [4]:
# Define our expected output structure as pydantic model

class Label(str, Enum):
  positive = "positive"
  negative = "negative"


class Prediction(BaseModel):
  prediction: Label

In [5]:
@text.prompt
def get_prompt(instructions: str, input_text: str, json: callable, Prediction, examples: Optional[tuple] = None):
  """Instructions: {{ instructions }}

  {% if examples %}
  {% for example in examples %}
  Text: {{ example[0] }}
  {{ json.dumps(Prediction(prediction=example[1]).model_json_schema()) }}
  {% endfor %}
  {% endif %}

  Text: {{ input_text}}\n
  """


get_prompt = partial(get_prompt, json=json, Prediction=Prediction)

In [6]:
# fix instructions for sentiment classifier prompt template

instructions = "You are a sentiment classifier. Your task is to classify whether the text below has positive or negative sentiment."
get_sentiment_classifier_prompt = partial(get_prompt, instructions=instructions)

In [7]:
# create a few static examples
examples = [('... the movie is just a plain old monster . ', 'negative', 'escaping the studio , piccoli is warmly affecting and so is this adroitly minimalist movie . ', 'positive')]

In [8]:
# run inference without examples

input = "it's a charming and often affecting journey."  # true label = "positive"

prompt = get_sentiment_classifier_prompt(input_text=input)

classification = generate.json(model=model, schema=Prediction)(prompt)
json.loads(classification)

{'prediction': 'positive'}

In [9]:
# run inference with examples

input = "it's a charming and often affecting journey."  # true label = "positive"

prompt = get_sentiment_classifier_prompt(input_text=input, examples=examples)

classification = generate.json(model=model, schema=Prediction)(prompt)
json.loads(classification)

{'prediction': 'positive'}

In [10]:
# run inference with examples on another input text

input = "the movie fails to live up to the sum of its parts . " # true label = negative

prompt = get_sentiment_classifier_prompt(input_text=input, examples=examples)

classification = generate.json(model=model, schema=Prediction)(prompt)
json.loads(classification)

{'prediction': 'negative'}